In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## MULTILAYER PERCEPTROM

In [3]:
cities = open("cities.txt", "r").read().splitlines()
cities[:8]

['dubai',
 'sharjah',
 'ajman',
 'zaranj',
 'taloqan',
 'shahrak',
 'maymana',
 'kushk']

In [4]:
len(cities)

14117

In [5]:
chars = sorted(list(set("".join(cities))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i : s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3
X, Y = [], []
for city in cities:
    context = [0] * block_size
    for ch in city + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([119858, 3]), torch.int64, torch.Size([119858]), torch.int64)

In [8]:
def build_dataset(cities):
    block_size = 3
    X, Y = [], []
    for city in cities:
        context = [0] * block_size
        for ch in city + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(cities)

n1 = int(.8 * len(cities))
n2 = int(.9 * len(cities))
n1, n2

Xtr, Ytr = build_dataset(cities[:n1])
Xdev, Ydev = build_dataset(cities[n1:n2])
Xte, Yte = build_dataset(cities[n2:])

In [9]:
C = torch.randn((27, 2))

In [10]:
emb = C[X]
emb.shape

torch.Size([119858, 3, 2])

In [11]:
W1 = torch.randn((6, 100)) # 6 -> 3 * 2
b1 = torch.randn(100)

In [12]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # tanh makes all values between -1 and 1, -1 infers
h

tensor([[ 0.9603,  1.0000,  0.9972,  ..., -0.9594, -0.9457,  0.9945],
        [ 0.9970,  1.0000, -0.9993,  ..., -0.9798, -0.8832,  0.9997],
        [ 0.9959,  1.0000,  0.4313,  ...,  0.9961,  0.9997, -0.7547],
        ...,
        [ 0.7950,  0.8921,  0.9966,  ..., -0.4197, -0.9057, -0.3924],
        [ 0.8950,  0.9985,  0.7593,  ..., -0.9700, -0.9588,  0.6516],
        [ 0.9854,  0.9998, -0.9795,  ..., -0.6592, -0.9542,  1.0000]])

In [13]:
W2 = torch.randn((100, 27)) # 6 -> 3 * 2
b2 = torch.randn(27)

In [14]:
logits = h @ W2 + b2
logits.shape
logits

tensor([[  4.0468,  -3.6988,  11.1589,  ..., -18.4666,  -1.0309,  -3.1405],
        [  9.5432,   5.1817,   8.4674,  ...,  -9.9981,  16.2116,  -0.6137],
        [  6.4522,  15.8746,  21.4495,  ..., -18.2408,   2.8403,  -1.2025],
        ...,
        [ -8.0051,  -8.5230,  11.5899,  ..., -15.4006,  -7.7747,  -0.0410],
        [  2.5339,  -0.2041,  12.9070,  ..., -11.6528,  -1.0072,  -2.9616],
        [  6.3493,   1.5614,   8.8954,  ...,  -2.6545,   9.2675,   2.4418]])

In [15]:
counts = logits.exp() # gets fake counts

In [16]:
prob = counts / counts.sum(1, keepdims=True)

In [17]:
Xtr.shape, Ytr.shape

(torch.Size([95858, 3]), torch.Size([95858]))

In [18]:
g = torch.Generator().manual_seed(23456)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2= torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [19]:
sum(p.nelement() for p in parameters)

11897

In [20]:
for p in parameters:
    p.requires_grad = True

In [24]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre
lri = []
lossi = []
stepi= []


In [ ]:

for i in range(200000):
    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (35,))

    #forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(35), Y].log().mean()
    ## below is equivalent
    loss = F.cross_entropy(logits, Ytr[ix])
    # backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track learning rates
    lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())



In [ ]:
plt.plot(stepi, lossi)
#10**-1 optimal learning rate

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:, 1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i, 0].item(), C[i, 1].item(), itos[i], ha="center", va="center", color="white")

In [ ]:
g = torch.Generator().manual_seed(23456)
for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print("".join(itos[i] for i in out))